In [2]:
#notes on using sklearn uses median age of inventory as data
import pandas as pd
pd.options.display.max_columns = 9999
pd.options.display.max_rows = 10
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
import pydotplus

#median age of currently listed homes
inventory = pd.read_csv('MedianAgeOfInventory_NSA_AllHomes_Metro.csv', encoding='latin')

In [ ]:
pd.options.display.max_rows = 999
inventory.head()

In [ ]:
test = inventory.loc[inventory['2019-10'].isnull()]
test

In [ ]:
#must remove null values for models to work
inventory1 = inventory.loc[inventory['2019-10'].notnull() & inventory['2019-11'].notnull() & 
                          inventory['2019-12'].notnull() & inventory['2020-01'].notnull()]
inventory1

In [ ]:
# 1. the value we want to predict
y = inventory1['2020-02']

In [ ]:
# 2. the input values used to generate model
x = inventory1[['2019-10','2019-11','2019-12','2020-01']]

In [ ]:
# 3. use DecisionTreeRegressor to generate model
inventory_model = DecisionTreeRegressor(random_state=1)
inventory_model.fit(x, y)

In [ ]:
# 4. use model to predict first 20 entries
print(inventory_model.predict(x.loc[:20]))

In [ ]:
# 5. measure how well the model fits actual data by generating mean_absolute_error
predicted_inventory = inventory_model.predict(x)
mean_absolute_error(y, predicted_inventory)


In [ ]:
# putting it all together - split data into x, y for training and x, y for calculating mean_absolute_error
train_x, val_x, train_y, val_y = train_test_split(x, y, random_state = 0)
# Define model
inventory_model = DecisionTreeRegressor()
# Fit model
inventory_model.fit(train_x, train_y)

# get predicted prices on validation data
val_predictions = inventory_model.predict(val_x)
print(mean_absolute_error(val_y, val_predictions))

In [ ]:
#puttin it all together even more: find model with nodes that fits the best
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

leaf_nodes = [5, 50, 500, 5000]
for max_leaf_nodes in leaf_nodes:
    my_mae = get_mae(max_leaf_nodes, train_x, val_x, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error: %d" %(max_leaf_nodes, my_mae))

In [ ]:
#last step is to use number of nodes that fit best to calculate prediction
final_model = DecisionTreeRegressor(max_leaf_nodes = 50, random_state=1)

# fit the final model
final_model.fit(x, y)
z = final_model.predict(x)
z

In [ ]:

export_graphviz(dtree, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
#use a different model: RandomForestRegressor this model uses many DecisionTreeRegressor and averages them out
train_x, val_x, train_y, val_y = train_test_split(x, y, random_state = 0)
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_x, train_y)
melb_preds = forest_model.predict(val_x)
print(mean_absolute_error(val_y, melb_preds))